In [1]:
import os

from ladybug.sql import SQLiteResult

from eppy import *
from eppy.modeleditor import IDF
from eppy.results import readhtml
from eppy.results import fasthtml


import pickle

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

import pprint

import numpy as np



In [2]:
root = os.getcwd()
model_dir = os.path.join(root, "group_models/Final_project_2-25-2022_3-1/run")
model_dir


'/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/group_models/Final_project_2-25-2022_3-1/run'

# IDF Adjustment

In [6]:
# # make a new dir for the output
adjusted_model_dir = os.path.join(root, "eppy_adjusted_models")
## NEED TO CHANGE THE LAST STRING TO CREATE NEW FOLDER!!!
new_dir_name = os.path.join(adjusted_model_dir, "02_28-3_change_analysis_per")
# os.makedirs(new_dir_name)

In [4]:
## UNCOMMENT TO ADD NEW VARIABLES TO THE IDF 

# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

# get the idf 
idfPath = os.path.join(model_dir, "in.idf")
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"
idf0 = IDF(idfPath, epw)

intVarsPath = os.path.join(root, "intVars")
with open(intVarsPath, "rb") as f:
    intVars = pickle.load(f)
intVars

# update the outputs with interesting variables 
currVars = [var.Variable_Name for var in idf0.idfobjects["Output:Variable"]]
for var in intVars:
    if var not in currVars:
        newObj = idf0.newidfobject("Output:Variable", Key_Value='', Variable_Name=var)

# change all reporting frequency
allVars = [var for var in idf0.idfobjects["Output:Variable"]]
for var in allVars:
    var.Reporting_Frequency = "Hourly"

allVars = [var for var in idf0.idfobjects["Output:Meter"]]
for var in allVars:
    var.Reporting_Frequency = "Monthly"

# change the run period
runPeriod = idf0.idfobjects["RunPeriod"][0]
runPeriod.Begin_Month = 1 
runPeriod.Begin_Day_of_Month = 1
runPeriod.End_Month = 12
runPeriod.End_Day_of_Month = 31

# remove extraneous electricity:facility terms 
# for some reason openstudio adds three of them which confuses
# the program that reads sql
extra_elect = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]
extra_elect


for e in extra_elect:
  idf0.removeidfobject(e)
check = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]
# should see a blank array showing these were removed
print(check)

# # save the updated idf there
# idf0.save(os.path.join(new_dir_name, "in2.idf"))

# # run the idf 
# idf0.run(output_directory=new_dir_name)

[]


# SQL (Simulation) Analyis

## Load SQL File

In [8]:
# access the sql file 
sqlFile = os.path.join(new_dir_name, "eplusout.sql")
sqld = SQLiteResult(sqlFile)

In [9]:
sqld.available_outputs_info

[{'output_name': 'Zone Windows Total Transmitted Solar Radiation Energy',
  'object_type': 'Zone',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'Site Outdoor Air Drybulb Temperature',
  'object_type': 'Zone',
  'units': 'C',
  'data_type': Temperature},
 {'output_name': 'Zone Windows Total Heat Gain Energy',
  'object_type': 'Zone',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'District Heating Hot Water Energy',
  'object_type': 'System',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'Surface Window Heat Loss Energy',
  'object_type': 'Zone',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'Surface Window Heat Gain Energy',
  'object_type': 'Zone',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'DistrictHeating:Facility',
  'object_type': 'Facility:DistrictHeating',
  'units': 'kWh',
  'data_type': Energy},
 {'output_name': 'District Cooling Chilled Water Energy',
  'object_type': 'System',
  'units': 'kWh',
  'data_type'

In [10]:

sqld.available_outputs

('Zone Windows Total Transmitted Solar Radiation Energy',
 'Site Outdoor Air Drybulb Temperature',
 'Zone Windows Total Heat Gain Energy',
 'District Heating Hot Water Energy',
 'Surface Window Heat Loss Energy',
 'Surface Window Heat Gain Energy',
 'DistrictHeating:Facility',
 'District Cooling Chilled Water Energy',
 'DistrictCooling:Facility',
 'District Cooling Chilled Water Rate',
 'Surface Heat Storage Energy',
 'Zone Windows Total Heat Loss Energy',
 'Zone Operative Temperature',
 'District Heating Hot Water Rate',
 'Electricity:Facility',
 'Zone People Total Heating Energy',
 'Zone Air Temperature',
 'Electricity:Facility',
 'Surface Average Face Conduction Heat Transfer Energy',
 'Zone Mean Radiant Temperature',
 'Zone Mean Air Temperature',
 'Electricity:Facility',
 'Zone Air Relative Humidity')

## Heating

In [11]:
facility_heating_collection = sqld.data_collections_by_output_name("DistrictHeating:Facility")
facility_heating_data = facility_heating_collection[-1]
facility_heating_monthly = facility_heating_data
facility_heating_monthly

Monthly Collection
Jan to Dec
Energy (kWh)
...12 values...

In [12]:
fig = go.Figure(data=go.Scatter(x=facility_heating_monthly.datetime_strings, y=facility_heating_monthly.values, mode='lines'))

fig.update_layout(
    title="Total Monthly Facility Heating Data",
    xaxis_title="Months",
    yaxis_title="Energy [kWh]",
)
fig.show()



## Cooling

In [13]:
cooling_collection = sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1]

cooling_monthly = cooling_collection


fig = go.Figure(data=go.Scatter(x=cooling_monthly.datetime_strings, y=cooling_monthly.values, mode='lines'))

fig.update_layout(
    title="Total Monthly Cooling Data",
    xaxis_title="Months",
    yaxis_title="Energy [kWh]",
)
fig.show()
cooling_monthly

Monthly Collection
Jan to Dec
Energy (kWh)
...12 values...

## Electricity

In [14]:
# read html 
html_path = os.path.join(new_dir_name, "eplustbl.htm")
# open file 
html_file = open(html_path, 'r').read()
# read the tables
htables = readhtml.titletable(html_file) 
# names of all tables
alltitles = [htable[0] for htable in htables]
# alltitles

In [15]:
filehandle = open(html_path, 'r') # get a file handle to the html file
namedtable = fasthtml.tablebyname(filehandle, "Custom Monthly Report")
pp = pprint.PrettyPrinter()
pp.pprint(namedtable)


['Custom Monthly Report',
 [['',
   'INTERIORLIGHTS:ELECTRICITY [J]',
   'EXTERIORLIGHTS:ELECTRICITY []',
   'INTERIOREQUIPMENT:ELECTRICITY [J]',
   'EXTERIOREQUIPMENT:ELECTRICITY []',
   'FANS:ELECTRICITY [J]',
   'PUMPS:ELECTRICITY [J]',
   'HEATING:ELECTRICITY []',
   'COOLING:ELECTRICITY []',
   'HEATREJECTION:ELECTRICITY []',
   'HUMIDIFIER:ELECTRICITY []',
   'HEATRECOVERY:ELECTRICITY []',
   'WATERSYSTEMS:ELECTRICITY []',
   'COGENERATION:ELECTRICITY []',
   'REFRIGERATION:ELECTRICITY []'],
  ['January',
   11610400000.0,
   0.0,
   43668900000.0,
   0.0,
   1637040000.0,
   204210000.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  ['February',
   10593700000.0,
   0.0,
   39617800000.0,
   0.0,
   1383680000.0,
   132063000.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  ['March',
   12041800000.0,
   0.0,
   44518300000.0,
   0.0,
   1695240000.0,
   215910000.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  ['Ap

In [18]:
data = namedtable[1][1:-1]
columns = namedtable[1][0]
e_monthly = pd.DataFrame(data=data, columns=columns)
e_monthly = e_monthly.drop(labels=[12,13,14])
e_monthly 

,,INTERIORLIGHTS:ELECTRICITY [J],EXTERIORLIGHTS:ELECTRICITY [],INTERIOREQUIPMENT:ELECTRICITY [J],EXTERIOREQUIPMENT:ELECTRICITY [],FANS:ELECTRICITY [J],PUMPS:ELECTRICITY [J],HEATING:ELECTRICITY [],COOLING:ELECTRICITY [],HEATREJECTION:ELECTRICITY [],HUMIDIFIER:ELECTRICITY [],HEATRECOVERY:ELECTRICITY [],WATERSYSTEMS:ELECTRICITY [],COGENERATION:ELECTRICITY [],REFRIGERATION:ELECTRICITY []
0,January,11610400000.0,0.0,43668900000.0,0.0,1637040000.0,204210000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,February,10593700000.0,0.0,39617800000.0,0.0,1383680000.0,132063000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,March,12041800000.0,0.0,44518300000.0,0.0,1695240000.0,215910000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,April,10828600000.0,0.0,41354800000.0,0.0,1722900000.0,306763000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,May,12041800000.0,0.0,44518300000.0,0.0,1767610000.0,293063000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,June,11559100000.0,0.0,42884800000.0,0.0,1984680000.0,532333000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,July,11311300000.0,0.0,42988300000.0,0.0,1982550000.0,538224000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,August,12041800000.0,0.0,44518300000.0,0.0,2235550000.0,671091000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,September,11260100000.0,0.0,42204200000.0,0.0,2221420000.0,556080000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,October,11610400000.0,0.0,43668900000.0,0.0,2204550000.0,382275000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
e_sum = e_monthly[e_monthly.columns[1:]].sum(1)
fig = go.Figure(data=go.Scatter(x=cooling_monthly.datetime_strings, y=e_sum, mode='lines'))

fig.update_layout(
    title="Total Monthly Electricity Data",
    xaxis_title="Months",
    yaxis_title="Energy [J]",
)
fig.show()
cooling_monthly

Monthly Collection
Jan to Dec
Energy (kWh)
...12 values...

## Heat Gain by Surface Window 

In [20]:
s_wind = sqld.data_collections_by_output_name_run_period('Surface Window Heat Gain Energy', 7)
s_wind_sum = [np.sum(d.values) for d in s_wind]
s_wind_sum #kwh



[341.9982069015349, 14665.322134559985, 14656.652156409014, 340.84307041726004]

In [21]:
windows = [(s.Name, s.area) for s in idf0.idfobjects['FenestrationSurface:Detailed']]
windows #m2

[('Sub Surface 1', 8.17546752000008),
 ('Sub Surface 2', 61.31600639999994),
 ('Sub Surface 3', 61.316006399999935),
 ('Sub Surface 4', 8.17546752000008)]

In [22]:
# todo normalize by area
windows_heat_gain = [(win[0], heat/win[1]) for win, heat in zip(windows, s_wind_sum)]
windows_heat_gain #kwh / m2 

[('Sub Surface 1', 41.8322507018573),
 ('Sub Surface 2', 239.17608134635458),
 ('Sub Surface 3', 239.0346830613063),
 ('Sub Surface 4', 41.69095768326858)]

## Heat Gain by Zone

In [23]:
solar_collection = sqld.data_collections_by_output_name_run_period("Zone Windows Total Heat Gain Energy", 7)
solar_by_zone = [np.sum(d.values) for d in solar_collection ]
solar_by_zone
# cooling_monthly = cooling_collection

[0.0, 0.0, 0.0, 14882.734298637732, 0.0, 14872.749640193622]

In [24]:
zone_cooling = sqld.tabular_data_by_name("Cooling Sizing Factor Information")
zones = [i[0].replace("Zone THERMAL ZONE: ", "") for i in np.array(list(zone_cooling.values()))][1:]
zones

['SPACE 101', 'SPACE 102', 'SPACE 103', 'SPACE 201', 'SPACE 202', 'SPACE 203']

In [25]:
fig = go.Figure([go.Bar(x=zones, y=solar_by_zone)])
fig.show()

# Compare Simulation and Historical Data

In [26]:
# load data 
historical_data_path = os.path.join(root, "data/Thornton Utilities 2019.xlsx")
hist_elect_hourly = pd.read_excel(historical_data_path, sheet_name="Electricity hourly", usecols="A:E")
hist_chw_hourly = pd.read_excel(historical_data_path, sheet_name="CHW hourly", usecols="A:E")
hist_hw_hourly = pd.read_excel(historical_data_path, sheet_name="HW hourly", usecols="A:E")
hist_all_monthly = pd.read_excel(historical_data_path, sheet_name="Monthly", usecols="A:E", skiprows=[0,1])

In [27]:
hist_all_monthly.columns

Index(['Month_Yr', 'Month', 'Chilled Water (ton-hrs)', 'Electricity (kWh)',
       'Hot Water (kBtu)'],
      dtype='object')

## Heating

In [28]:
kbtu_kwHr = 1/3412 # 1 kbtu = x kwHr
months = facility_heating_monthly.datetime_strings

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, 
    y=hist_all_monthly["Hot Water (kBtu)"], 
    mode='lines+markers',
    name="Historical Data"))

fig.update_layout(
    title="Historical Monthly Hot Water ",
    xaxis_title="Months",
    yaxis_title="Energy (kbtu)",
)
fig.show()

In [29]:
kwh_mbtu = 3.412/1000 # 1 kwh = 3.412141 Mbtu;
months = facility_heating_monthly.datetime_strings

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, 
    y=np.array(facility_heating_monthly.values)*kwh_mbtu, 
    mode='lines+markers',
    name="Historical Data"))

fig.update_layout(
    title="Simulation Monthly Hot Water",
    xaxis_title="Months",
    yaxis_title="Energy (mbtu)",
)
fig.show()

In [30]:
1/3.412

0.29308323563892147

In [31]:
kbtu_kwHr = 1/3.412 # 1 kbtu = x kwHr
months = facility_heating_monthly.datetime_strings

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, 
    y=hist_all_monthly["Hot Water (kBtu)"]*kbtu_kwHr , 
    mode='lines+markers',
    name="Historical Data"))

fig.add_trace(go.Scatter(x=months, 
    y=facility_heating_monthly.values, # kwh
    mode='lines+markers',
    name="Simulation"))

fig.update_layout(
    title="Total Monthly Hot Water",
    xaxis_title="Months",
    yaxis_title="Energy (kwHr)",
)
fig.show()

In [34]:
hist_all_monthly["Hot Water (kBtu)"]*kbtu_kwHr

0      9551.944966
1     11205.168864
2     10021.888756
3      5739.921388
4      8015.771756
5      2812.879287
6      1998.993427
7      1290.168084
8      1966.378451
9      4721.300882
10     7453.441033
11    10523.656309
Name: Hot Water (kBtu), dtype: float64

## Cooling

In [32]:
# plot
tonHr_kwHr = 3.5169 # 1 ton = x kwHr

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, 
    y=hist_all_monthly["Chilled Water (ton-hrs)"]*tonHr_kwHr, 
    mode='lines+markers',
    name="Historical Data"))

fig.add_trace(go.Scatter(x=months, 
    y=cooling_monthly.values,
    mode='lines+markers',
    name="Simulation"))

fig.update_layout(
    title="Total Monthly Chilled Water ",
    xaxis_title="Months",
    yaxis_title="Energy (kwHr)",
)
fig.show()

## Electricity

In [33]:
j_kwHr = 2.77778e-7 # 1 ton = x kwHr

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, 
    y=hist_all_monthly["Electricity (kWh)"],
    mode='lines+markers',
    name="Historical Data"))

fig.add_trace(go.Scatter(x=months, 
    y=e_sum*j_kwHr,
    mode='lines+markers',
    name="Simulation"))

fig.update_layout(
    title="Total Monthly Electricity",
    xaxis_title="Months",
    yaxis_title="Energy (kwHr)",
)
fig.show()